In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def load_model(model_path):
    # model = ElectraForSequenceClassification.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    # tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
    # tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
    return model, tokenizer

model_path = '/content/drive/MyDrive/TalkMoves-main/baseline_public/deberta-v3-xsmall.pth'  # Adjust to your model's path
model, tokenizer = load_model(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
from torch.quantization import quantize_dynamic

# Specify the parts of the model you want to dynamically quantize.
# Typically, for transformers, we quantize the linear layers since they dominate the model size.
model_quantized = quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [ ]:
import os
def print_model_size(model, label):
    torch.save(model.state_dict(), "temp.p")
    print(f'{label} Model Size (MB):', os.path.getsize("temp.p") / 1e6)
    os.remove("temp.p")

print_model_size(model, "Original")
print_model_size(model_quantized, "Quantized")


Original Model Size (MB): 283.398836
Quantized Model Size (MB): 219.305872


In [ ]:
text = "Hello, world!"
inputs = tokenizer(text, return_tensors="pt")

# Use the original and quantized models to make predictions
with torch.no_grad():
    original_output = model(**inputs)[0]
    quantized_output = model_quantized(**inputs)[0]

print("Original Model Output:", original_output)
print("Quantized Model Output:", quantized_output)


Original Model Output: tensor([[ 6.6016,  0.8905, -4.1772, -4.5435, -3.9734, -2.9081, -8.6221]])
Quantized Model Output: tensor([[ 6.6141,  0.9206, -4.1198, -4.6764, -4.1265, -2.8396, -8.6221]])


In [ ]:
# Specify the path where you want to save the model
model_save_path = '/content/drive/MyDrive/TalkMoves-main/baseline_public/quantized_bert_model.pth'

# Save the quantized model
torch.save(model_quantized.state_dict(), model_save_path)

print(f'Model successfully saved to {model_save_path}')


Model successfully saved to /content/drive/MyDrive/TalkMoves-main/baseline_public/quantized_bert_model.pth


In [ ]:
# Save the entire quantized model, not just the state_dict
model_save_path = '/content/drive/MyDrive/TalkMoves-main/quantized_models/quantized_deberta-v3-xsmall.pth'
torch.save(model_quantized, model_save_path)

In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch
from torch.quantization import quantize_dynamic

model_path = '/content/drive/MyDrive/TalkMoves-main/baseline_public/teacher_electra_base.pth'

# Specify the path where you want to save the model
#model_save_path = '/content/drive/MyDrive/TalkMoves-main/baseline_public/quantized_electra_model.pth'

# Define the model architecture again
model_loaded = ElectraForSequenceClassification.from_pretrained(model_path)

# Load the quantized state dict
model_loaded.load_state_dict(torch.load(model_save_path))

# Apply dynamic quantization again (necessary because we are reloading the architecture)
model_loaded = quantize_dynamic(model_loaded, {torch.nn.Linear}, dtype=torch.qint8)

print('Model loaded and quantized successfully.')



/usr/local/lib/python3.10/dist-packages/torch/_utils.py:383: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


TypeError: Expected state_dict to be dict-like, got <class 'transformers.models.electra.modeling_electra.ElectraForSequenceClassification'>.

In [ ]:
import torch

# Path to the saved model
model_path = '/content/drive/MyDrive/TalkMoves-main/quantized_models/quantized_electra_model.pth'

# Load the quantized model
quantized_model = torch.load(model_path)

# Put the model in evaluation mode
quantized_model.eval()

print("Quantized model loaded successfully and is ready for inference.")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:383: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


Quantized model loaded successfully and is ready for inference.


In [ ]:
# fp-16

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load your pre-trained model
model_path = '/content/drive/MyDrive/TalkMoves-main/baseline_public/deberta-v3-xsmall.pth'  # Update to your model path
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to('cuda')
model.eval()

# Convert model parameters to FP16
model.half()  # This converts all floating point parameters and buffers to half precision

# Save the FP16 model
model_save_path = '/content/drive/MyDrive/TalkMoves-main/fp_16/deberta-v3-xsmall.pth'
torch.save(model, model_save_path)
